# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- *сalls* — количество звонков,
- *minutes* — суммарная длительность звонков в минутах,
- *messages* — количество sms-сообщений,
- *mb_used* — израсходованный интернет-трафик в Мб,
- *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Откройте и изучите файл

Загрузим данные и посмотрим на них. По заданию предобработка не нужна.

In [2]:
df = pd.read_csv('datasets/users_behavior.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Количество звонков и количество смс-сообщений по смыслу должно быть int. Преобразуем тип.

In [3]:
df['calls'] = df['calls'].astype('int')
df['messages'] = df['messages'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


Посмотрим, какой процент объектов в датасете относится к пользователям тарифа "Ультра", т.е. значение is_ultra = 1. Для этого количество "единиц" нужно разделить на количество объектов. В данном случае количество "единиц" - это сумма значений столбца is_ultra. Сумма значений, делённая на их количество - по определению среднее арифметическое. 

In [4]:
print('Процент объектов "Ультра" в датасете: {:.2%}'.format(df['is_ultra'].mean()))

Процент объектов "Ультра" в датасете: 30.65%


**Вывод**

Загружен датасет с 3214 объектами, из которых к тарифу "Ультра" относятся 30.65%. Тип данных столбцов calls и messages преобразован в int.

## Разбейте данные на выборки

Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Функция train_test_split делит датасет только на две части, поэтому воспользуемся ей дважды: на первом шаге выделим 60% на обучающую часть, а на втором - разделим оставшиеся 40% пополам: на валидационную и тестовую выборки.

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=42
)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=0.5, random_state=42
)
print('Рамер обучающей выборки:', features_train.shape, target_train.shape)
print('Рамер валидационной выборки:', features_valid.shape, target_valid.shape)
print('Рамер тестовой выборки:', features_test.shape, target_test.shape)
print()
print('Процент объектов "Ультра" в обучающей выборке: {:.2%}'.format(target_train.mean()))
print('Процент объектов "Ультра" в валидационной выборке: {:.2%}'.format(target_valid.mean()))
print('Процент объектов "Ультра" в тестовой выборке: {:.2%}'.format(target_test.mean()))

Рамер обучающей выборки: (1928, 4) (1928,)
Рамер валидационной выборки: (643, 4) (643,)
Рамер тестовой выборки: (643, 4) (643,)

Процент объектов "Ультра" в обучающей выборке: 30.76%
Процент объектов "Ультра" в валидационной выборке: 30.64%
Процент объектов "Ультра" в тестовой выборке: 30.33%


**Вывод**

Данные разделены на обучающую, валидационную и тестовую выборки, доля объектов с тарифом "Ультра" в которых сопоставима с долей таких объектов во всём датасете.

## Исследуйте модели

В качестве первой модели рассмотрим решающее дерево. Переберём варианты глубины дерева от 1 до 10 и найдём лучшую модель - ту, у которой доля правильных ответов на валидационной выборке больше.

In [6]:
accuracy_tree = 0

for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth = depth, random_state=42)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > accuracy_tree:
        model_tree = model
        accuracy_tree = accuracy
        depth_tree = depth
        
print(
    'Доля правильных ответов на валидационной выборке при решающем дереве: {:.2f}'
    .format(accuracy_tree)
)
print('Глубина дерева:', depth_tree)

Доля правильных ответов на валидационной выборке при решающем дереве: 0.80
Глубина дерева: 8


У дерева решений с глубиной 8 accuracy на валидационной выборке 0.8, что больше 0.75. Это хорошая модель, которую можно использовать. Но попробуем найти лучшее решение - используем модель случайного леса. Переберём 10 вариантов леса с количеством деревьев от 3 до 30 с шагом 3. Для каждого леса переберём варианты максимальной глубины дерева от 1 до 10. Итого это 100 моделей. Найдём, какая из них даст большую долю правильных ответов на валидационной выборке.

In [7]:
accuracy_forest = 0

for estimators in range(3, 31, 3):
    for depth in range(1, 11):
        model = RandomForestClassifier(n_estimators = estimators, max_depth = depth, random_state=42)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > accuracy_forest:
            model_forest = model
            accuracy_forest = accuracy
            estimators_forest = estimators
            depth_forest = depth
            
print(
    'Доля правильных ответов на валидационной выборке при случайном лесе: {:.2f}'
    .format(accuracy_forest)
)
print('Количество деревьев в модели:', estimators_forest)
print('Максимальная глубина дерева:', depth_forest)

Доля правильных ответов на валидационной выборке при случайном лесе: 0.82
Количество деревьев в модели: 24
Максимальная глубина дерева: 9


Модель случайного леса с 24 дереьями показала лучшие результаты, чем модель с деревом решений. Рассмотрим логистическую регрессию. Возможно, она будет давать результаты не хуже, но работать быстрее.

In [8]:
model_regression = LogisticRegression(random_state=42, solver='liblinear')
model_regression.fit(features_train, target_train)
accuracy_regression = model_regression.score(features_valid, target_valid)
print(
    'Доля правильных ответов на валидационной выборке при логистической регрессии: {:.2f}'
    .format(accuracy_regression)
)

Доля правильных ответов на валидационной выборке при логистической регрессии: 0.72


У модели логистической регрессии accuracy на валидационной выборке меньше 0.75 - такая модель нам не подходит.

**Вывод**

Рассмотрены 10 моделей решающего дерева, 100 моделей случайного леса и модель логистической регрессии. Лучшие результаты показала модель случайного леса с 24 деревьями и максимальной глубиной дерева 9. На валидационной выборке accuracy = 0.82. Наихудшие показатели дала модель логистической регрессии: значение accuracy на валидационной выборке не преодолело планку 0.75.

## Проверьте модель на тестовой выборке

Проверим, даёт ли выбранная модель такие же хорошие результаты на тестовой выборке.

In [9]:
print(
    'Доля правильных ответов на тестовой выборке при случайном лесе: {:.2f}'
    .format(model_forest.score(features_test, target_test))
)

Доля правильных ответов на тестовой выборке при случайном лесе: 0.82


**Вывод**

Модель model_forest показала себя на тестовой выборке так же хорошо, как и на валидационной.

## (бонус) Проверьте модели на адекватность

Сравним accuracy нашей модели с долей правильных ответов, которые получались бы, если бы модель выбирала ответ случайно, а не опираясь на features. Т.к. модель обучалась на обучающей выборке, будем считать вероятность получить значение 1 ("Ультра") по target_train. Для этого количество "единиц" нужно разделить на общее количество объектов в обучающей выборке. В данном случае это равносильно вычислению среднего значения. Создадим объект Series того же размера, что и target_valid, и заполним его значениями 1 с вычисленной вероятностью - это будет предполагаемый прогноз модели, которая опиралась бы на случайный выбор. Сравним такое предсказание с target_valid и вычислим accuracy. Такой прогноз может отклоняться ближе и дальше от target_valid, поэтому повторим эксперимент со случайным предсказанием 100 раз и найдём средние значения.

In [10]:
ultra_objects = 0
accuracy_mean = 0

p = np.full(len(target_valid), target_train.mean()) #p - вектор одинаковых вероятностей

for _ in range(100):
    predictions = pd.Series(np.random.binomial(1, p), index = target_valid.index)
    ultra_objects += predictions.mean()
    accuracy_mean += accuracy_score(target_valid, predictions)
    
ultra_objects /= 100
accuracy_mean /= 100

print(
    'Средний процент объектов "Ультра" в валидационной выборке при случайном выборе: {:.2%}'
    .format(ultra_objects)
)
print(
    'Средняя доля правильных ответов на валидационной выборке при случайном выборе: {:.2f}'
    .format(accuracy_mean)
)

Средний процент объектов "Ультра" в валидационной выборке при случайном выборе: 30.46%
Средняя доля правильных ответов на валидационной выборке при случайном выборе: 0.57


**Вывод**

При случайном выборе среднее значение accuracy приблизительно 57%. Можно считать нашу модель адекватной - она работает значительно лучше, чем случайность.

## Вывод

Датасет, содержащий информацию о поведении клиентов и о тарифах, которыми они пользуются, был разделён на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. На обучающей выборке было обучено 10 моделей решающего дерева, 100 моделей случайного леса и модель логистической регрессии. Лучшее значение доли правильных ответов (0.82) на валидационной выборке показала модель случайного леса с 24 деревьями и максимальной глубиной дерева 9. На тестовой выборке эта модель дала ту же долю правильных ответов, что и на валидационной. Также модель прошла проверку на адекватность: она даёт значительно больше правильных ответов, чем при случайном выборе тарифа.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
